# Using MNE-Python for frequency and time-frequency analysis

    Authors: Britta Westner, partly based on MNE-Python tutorials and examples
    License: BSD (3-clause)

Let's dive into MNE-Python! We will get to know MNE-Python's way of working by conducting some frequency and time-frequency decompositions.

In the first part, we will explore some parts of MNE-Python together. In later parts, you will get to answer questions (exploring the data and code) and do some exercises (fixing and writing code).

I hope your MNE-Python journey won't end there - and you'll go explore our tutorials and examples as well!

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import mne
from mne.datasets import somato

Above, we imported the packages we will be using. MNE-Python, of course, but also NumPy, SciPy, and Matplotlib. As you can see, we are again using shorthand for some of them, e.g., importing the module `pyplot` from Matplotlib as `plt`- so we have to write only `plt.plot()` instead of `matplotlib.pyplot.plot()`. 

And we are doing the same for `mne.datasets.somato`, which we shorten to `somato` - more about this dataset below!

Lastly, we added `%matplotlib inline` to force our figures to be embedded into the notebook by default.

## The dataset

For our MNE-Python adventure, we are using a dataset that is available from within MNE-Python. If you followed the setup instructions, you will have already downloaded it. If not - don't worry, the code below will prompt MNE to download the data _if it has not been downloaded yet_. 

The dataset is called `somato`, referring to the paradigm: somatosensory stimulation was applied to elicit event-related (de-)synchronization in the beta-band.

In [ ]:
# mne.set_log_level('error')  # suppress some of MNE's output to keep the notebook slim

data_path = somato.data_path(download=True)  # download the data if necessary

# construct the path to the raw file
raw_fname = data_path / 'sub-01/meg/sub-01_task-somato_meg.fif'

## Read the data and inspect the raw file

Let's start with the raw data. The data is MEG data from an Elekta system. These files are also known as fif-Files, and we can read them in with the fif-reader of MNE-Python. 

MNE-Python has readers for an exhaustive list of systems! You can see (some) of them by starting to type `mne.io.read` in the code cell below and see what autocompletion gives you. - A neat trick for when you have trouble fully remembering a function name!

A more exhaustive way is to consult the **API Reference** on the MNE-Python webpage: https://mne.tools/stable/api/file_io.html


In [ ]:
# try out what autocompletion gives you for mne.io.read
# start typing! - depending on your system/IDE you might also have to hit tab for completion


In [ ]:
# read the raw file
raw = mne.io.read_raw_fif(raw_fname)

We can look at the so-called info object that is part of the `Raw` object. This has handy information about your data!

In [ ]:
raw.info

And we can of course plot the data. Here, you could for example also mark bad trials and channels (to which we will take a simple automated approach today). The data browser should open in a new window or be operable within the notebook - however, this is not fool-proof with Jupyter Notebooks. Another reason to not rely on them too strictly throughout your data analysis!

In [ ]:
raw.plot()

In [ ]:
# look at what changed in raw
raw.info

In [ ]:
raw.annotations

In [ ]:
# if you messed with raw, it's a good idea to reload it :)
raw = mne.io.read_raw_fif(raw_fname)

## Epoch the data

Time to learn how to epoch the data into trials!

For that we of course need triggers - that is quite an easy setup here: we have only one trigger value and one trigger channel. The `event_id` (the trigger value) is 1.

We will cut fairly long epochs: from -1 to 3 seconds around the event. We will already baseline-correct our events with a baseline extending to 0.

Another thing we want to set up is which channels to keep when epoching: we will keep only the MEG gradiometers (the system has magnetometers, too!) and the EOG (which we will use for data cleaning).

And finally, we use some simple heuristic to disregard noisy trials, based on cut-off values.

In [ ]:
# setting up all our parameters for epoching

# identify events and specify epoching parameters
events = mne.find_events(raw, stim_channel='STI 014')
event_id, tmin, tmax = 1, -1., 3.
baseline = (None, 0)

# specify which channels to keep after epoching
picks = mne.pick_types(raw.info, meg='grad', eeg=False, eog=True, stim=False)

# rejecting trials based on a simple heuristic
rejecting = dict(grad=4000e-13, eog=350e-6)

In [ ]:
# We can look at the events array:
events

Now for the actual epoching - we give all this information to the `Epochs` class - it will make our `epochs` object!

In [ ]:
epochs = mne.Epochs(
    raw,
    events,
    event_id, 
    tmin,
    tmax,
    picks=picks,
    baseline=baseline,
    reject=rejecting,
    preload=True
)

In [ ]:
epochs

## Some MNE-Python idiosyncracies

Let's dive into some of the things to be aware of when working with Python and MNE-Python!

Let's first make a _copy_ of our epochs - it will become clear why :)

We can do this with a _method_ that belongs to the `epochs` object.

In [ ]:
test_epochs = epochs.copy()

MNE-Python makes it possible to chain methods:

<div class="alert alert-warning">
    <b>QUESTION</b>:
     <ul>
    <li> Look at the code below: what operations do you expect to happen and in which order?</li>
    </ul>
</div>

**Answer**: 

In [ ]:
test_epochs.filter(l_freq=None, h_freq=15.).average().plot();

<div class="alert alert-warning">
    <b>QUESTION</b>:
     <ul>
    <li> What do you think happened to test_epochs?</li>
    </ul>
</div>

**Answer**: 

Well ... let's find out by plotting and comparing it to our original epochs!

In [ ]:
test_epochs.average().plot(titles='test epochs');
epochs.average().plot(titles='original epochs');

How can I find out which methods an object has?
- via tab / auto completion
- looking at the API reference of the class: https://mne.tools/stable/generated/mne.Epochs.html
- calling `dir()`

In [ ]:
dir(epochs)

... and how do I know if a method operates _in place_?!

Look at the documentation!
- either online or:
- by calling for help from within Python :)

In [ ]:
?epochs.filter

You can also find out whether an object has, e.g., been filtered by looking at it's print-out:

In [ ]:
test_epochs

## Frequency analysis

Now that we have covered some of the workings of MNE-Python, let's get to our actual task: frequency decomposition!

We start by exploring the frequency content of our epochs (without time-resolution): let's check out the **power spectrum**!

We choose the multitaper method here - the `bandwidth` parameter controls the spectral resolution (how many tapers). You can increase the resolution by choosing a narrower bandwidth at the cost of longer computation time.

In [ ]:
epochs_psd = epochs.compute_psd(method='multitaper', fmin=2., fmax=45., bandwidth=2.)

In [ ]:
# as always, we can get some insight by printing the object
epochs_psd

In [ ]:
# let's plot this - we force the notebook to open an extra, interactive figure
%matplotlib qt
epochs_psd.plot();

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li> What is the name of the EEG channel with the highest power at the alpha peak?</li>
    <li> Based on this plot, what do you expect the topography at 9 Hz to look like?</li>
    <li> Can you find a way to confirm this (without writing any code)?</li>
    </ul>
</div>

**Answers:**


We can also look at the topographies of this power spectrum, e.g. for different frequency bands. Here, we have to specify a channel type!

In [ ]:
%matplotlib inline
bands = {'Alpha (8-12 Hz)': (8, 12), 
         'Beta (12-30 Hz)': (12, 30), 
         'Gamma (30-45 Hz)': (30, 45)}
epochs_psd.plot_topomap(ch_type='grad', bands=bands, normalize=False, cmap='viridis');

## Time-frequency analysis: power and inter-trial coherence

Let's now compute time-frequency representations (TFRs) from our Epochs.
We can also look at power and inter-trial coherence (ITC).

To this we'll use the method `epochs.compute_tfr()`, asking it to use Morlet wavelets for the decomposition.

In [ ]:
# define frequencies of interest (log-spaced)
freqs = np.logspace(*np.log10([8, 45]), num=20)

# number of cycles 
n_cycles = 3.  

# compute the TFR using Morlet wavelets
power, itc = epochs.compute_tfr(
    method='morlet', 
    freqs=freqs, 
    n_cycles=n_cycles, 
    average=True,
    use_fft=True,
    return_itc=True, 
    decim=3, 
    n_jobs=1)

## Inspect time-frequency-resolved power


Let's look at the time-frequency spectra of all channels - in one plot!


<div class="alert alert-info"><h4>Note</h4><p>The generated figure is interactive. In the topo you can click
    on an image to visualize the data for one sensor.
    You can also select a portion in the time-frequency plane to
    obtain a topomap for a certain time-frequency region.</p></div>



Before we get there - let's also specify some parameters, namely: the baseline mode and length!

In [ ]:
# Some setting for our baseline, which will be applied to the plots
baseline_mode = 'logratio' 
baseline = (-0.75, -0.1)

<div class="alert alert-warning">
    <b>QUESTION</b>:
     <ul>
    <li> We are moving a 3 cycles long time window. Is our baseline a good choice?</li>
    </ul>
</div>

**Answer:** 

Now, let's finally plot this!

In [ ]:
%matplotlib qt
power.plot_topo(baseline=baseline, mode=baseline_mode, title='Average power');

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li> Hummm ... This looks different than the power plots above. What looks different?</li>
    <li> Can you find out what's going on by experimenting with the plotting parameters?</li>
    <li> Going back to the "good" plot, identify the channel with the strongest beta increase.</li>
    </ul>
</div>

**Answers:** 


Let's plot one channel of interest - the one you identified above! 

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li> Find the channel's index by completing the code below.</li>
    </ul>
</div>

In [ ]:
chan_idx = power.ch_names.index()  # complete this!

Let's plot the TFR of this one channel!

In [ ]:
%matplotlib inline
power.plot(chan_idx, baseline=baseline, mode=baseline_mode, title=power.ch_names[chan_idx]);

We can also look at topographies again! 

We have done this above for the power spectra. Our TFR output has a similar method, however, not all parameters are the same. Below, you find the code for plotting the **beta band topography**. Let's assume you also want to plot the following bands:
- alpha: 8-12 Hz
- gamma: 30-45 Hz

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li> Can you modify the code to plot all bands in a for-loop?</li>
    </ul>
</div>

Tips: 
- you can create a figure and axes object with `fig, axes = plt.subplots(1, 3, figsize=(7, 4))` and `power.plot_topomap()` takes an ax via the `axes` parameter
- you also have to suppress the immediate plotting of the figure (to not end up with only the first one): add `show=False` to your `power.plot_topomap()` call

In [ ]:
# code to plot beta band
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.0, fmin=13., fmax=30., 
                   baseline=baseline, mode=baseline_mode, contours=1, colorbar=False);

In [ ]:
# Answer: for loop code


## Joint Plots

Pretty neat are the creation of joint plots showing both the aggregated TFR
across all channels and topomaps at specified times and frequencies to obtain
a quick overview regarding oscillatory effects across time and space.



In [ ]:
power.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.5, 10.), (1., 15.)]);

# Inspect ITC

Oh, I almost forgot: we also computed the inter-trial coherence! Pretty good idea to check if we want to distinguish induced from evoked activity! This one will show us activity that is phase-locked across trials and can thus be assumed to be evoked!


In [ ]:
%matplotlib inline
itc.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.15, 10.)]);

<div class="alert alert-warning">
    <b>QUESTION</b>:
     <ul>
    <li> What can we conclude from this for our late beta effects?</li>
    </ul>
</div>

**Answer:** 

## Test the beta-band against baseline

Let's assume we now want to test if our post-stimulation period is actually significantly different from the baseline!

For that, we have to recompute our time-frequency representation to include single trials. This time, we can skip the ITC computation.

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li> Can you modify the TFR computation to give you single trials and not compute the ITC?</li>
    </ul>
</div>


In [ ]:
# Answer:
# careful, we only have one output now!


We express the data as percentage increase or decrease as compared to the baseline - that way, we can test the data against 0 later on.

In [ ]:
power.apply_baseline(baseline, mode='percent')

For cluster permutation tests, you have to identify a cluster threshold. MNE-Python can automatically compute that for you (if you would like a threshold that corresponds to a 0.05 alpha level). But let's for the fun of it compute it ourselves for a more rigorous threshold!

In [ ]:
aval = 0.001  # our alpha level
df = power.data.shape[0] - 1  # compute degrees of freedom
thresh = sp.stats.t.ppf(1 - aval, df)  # compute the corresponding t-value (one-tailed)

To keep computational time reasonable, we will not test all channels but instead choose one channel to test (our previously identified channel with the biggest power increase). 

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li> Look at the shape of the data: which axis is the channels dimension?</li>
    <li> Can you index the data in the permutation statistics function call with our channel?</li>
    </ul>
</div>

Tip: The channel index was called `chan_idx`.

In [ ]:
# Answer:

**Answer**:


We run 1000 permutations and a one-tailed test (we only test relative _increases_, no decreases).

**Remember to correctly index the data!**


In [ ]:
t_obs, clusters, p_vals, h0 = mne.stats.permutation_cluster_1samp_test(
    power.data[],  # ADD INDEXING HERE!
    threshold=thresh,
    n_permutations=1000,
    tail=1,
    out_type='mask',
    seed=1519  # add a seed to make results reproducible
)

## Plot the results

Now begins a bit of tedious work to be able to plot the data. 

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
     <li> How many clusters are found? Are all of them below 0.05? Inspect the output of the test function!
    <li> Look at the first code snippet below. Can you add comments to it that explain the code line by line?</li>
    </ul>
</div>

**Answer:**


In [ ]:
# Study the code and comment!

average_data = power.average().data[chan_idx, ::]

data_plot = np.nan * np.ones_like(average_data)

for cluster, p_val in zip(clusters, p_vals):
    if p_val < 0.05:       
        data_plot[cluster] = average_data[cluster]

Now we can plot the results - we choose to plot the original data in gray scale and overlay the parts belonging to a cluster in red.

We'll do this only using Matplotlib!

In [ ]:
# plot the original data
plt.imshow(
    average_data,
    extent=[power.tmin, power.tmax, power.freqs[0], power.freqs[-1]],
    origin='lower',
    aspect='auto',
    cmap='gray',
)

# plot the parts belonging to a cluster.
max_val = np.nanmax(abs(data_plot))
plt.imshow(
    data_plot,
    extent=[power.tmin, power.tmax, power.freqs[0], power.freqs[-1]],
    origin='lower',
    aspect='auto',
    cmap='RdBu_r',
    vmin=-max_val,
    vmax=max_val
)

plt.title('Channel: %s' % power.ch_names[chan_idx])
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')